In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ephesus.data import get_data_csv, get_data_json
from ephesus.sentence import return_label, load_model
from ephesus.duration import Duration
from ephesus.timedate import Date 
import numpy as np
import pandas as pd
from ephesus.duration import Duration
import re

2022-06-08 22:42:02.819531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 22:42:02.819637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [163]:
df = pd.DataFrame(get_data_csv()["translation"])

In [164]:
model = load_model("../models/model_full/model-best")

In [165]:
df

,translation
fichier,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ..."
...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...


In [166]:
df["entities"] = df["translation"].apply(lambda x: return_label(x, model))

In [167]:
def extract(prediction, label):
    for elm in prediction:
        if elm[1] == label:
            if elm[0] == np.nan:
                return np.nan
            else:
                return elm[0]  

In [168]:
df["raw_date"] = df["entities"].apply(lambda x: extract(x,'Date'))

In [169]:
df["Duration"] = df["entities"].apply(lambda x: extract(x,'Duration'))

In [170]:
def apply_date(sentence):
    if sentence == None:
        return None
    else: 
        df = Date().transform_data(sentence)
        return [df.iloc[0,2], df.iloc[0,1]]

In [171]:
test = 'le 10 juin'
temp = Date().transform_data(test)
temp

,data,day,month,year,day_of_week,day_from_today
0,"[10, juin]",10,6,99,99,99


In [172]:
apply_date(test)

[6, 10]

In [173]:
df["Date"] = df["raw_date"].apply(lambda x: apply_date(x))

In [174]:
df.drop(columns=["entities", "raw_date"], inplace=True)

In [175]:
df.head()

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[2, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None


In [111]:
Duration().explicit_duration("pendant 3 semaines")

{'CareDuration': ['3 semaines'],
 'CareDurationType': ['Weeks'],
 'CareEnd': [nan]}

In [109]:
test = Duration()

In [110]:
test.explicit_duration("pendant 3 jours")

{'CareDuration': ['3 jours'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}

In [19]:
df.iloc[0,2][0] = df.iloc[0,2][0] + 2 

In [20]:
df.iloc[0,2][0]

4

In [21]:
test.liste_mois

'(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)'

In [32]:
test.implicit_duration_1('jusqu\'au 24 juin')

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['24 juin']}

In [33]:
test.implicit_duration_1('jusqu\'au 24 juin',(6,10))

{'CareDuration': [14], 'CareDurationType': ['Days'], 'CareEnd': ['24 juin']}

In [34]:
test.get_duration(sentence='jusqu\'au 16 juin', CareBeginDate=[6,20])

{'CareDuration': [nan], 'CareDurationType': ['Days'], 'CareEnd': ['16 juin']}

In [27]:
liste_mois = '(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)'

In [28]:
re.search("(?<=jusqu\'au\s)(\w\s)*\d{1,2}\s"+liste_mois, 'jusqu\'au 16 juin')

<re.Match object; span=(9, 16), match='16 juin'>

In [35]:
test.get_duration(sentence="jusqu'au 16 juin")

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['16 juin']}

In [36]:
test.get_duration(sentence='pour 10 jours')

{'CareDuration': ['10'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}

In [37]:
def get_duration_df(x):
    if x is None:
        return None

In [39]:
test = pd.DataFrame()

In [40]:
type(test) == pd.core.frame.DataFrame

True

In [41]:
type(df["Duration"])

pandas.core.series.Series

In [42]:
duration = df["Duration"]
date = df["Date"]

In [46]:
df_temp = df.dropna()
df_temp

,translation,Duration,Date
fichier,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]"
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]"
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]"
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]"
...,...,...,...
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]"
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]"
3e7ffb67-8872-4172-bf18-4fe338ecae3f_80f9b3d2-47b8-4152-9e45-a26dcf173335,Injection sous-cutanée à faire une fois par jo...,pendant 30 jours,"[4, 7]"


In [47]:
test_1 = Duration().explicit_duration("pour 4 jours")

In [48]:
test_2 = Duration().explicit_duration("pendant 8 mois")

In [49]:
full_test = pd.DataFrame({'test':[test_1,test_2]})
full_test

,test
0,"{'CareDuration': ['4'], 'CareDurationType': ['..."
1,"{'CareDuration': ['8'], 'CareDurationType': ['..."


In [50]:
full_test['CareDuration'] = full_test["test"].apply(lambda x:x['CareDuration'][0])

In [51]:
full_test

,test,CareDuration
0,"{'CareDuration': ['4'], 'CareDurationType': ['...",4
1,"{'CareDuration': ['8'], 'CareDurationType': ['...",8


In [52]:
Duration().implicit_duration_1("jusqu'au 20 juin", [6,10],df=True)

,CareDuration,CareDurationType,CareEnd
0,10,Days,20 juin


In [53]:
df

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[4, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"


In [69]:
def test(x):
    if x is None:
        return None
    return Duration().implicit_duration_1(x)

In [70]:
df_temp = df.copy().dropna()

In [71]:
df_temp["test"] = df_temp["Duration"].apply(lambda x: test(x))

In [72]:
df_temp

,translation,Duration,Date,test
fichier,,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]",None
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]",None
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]",None
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]",None
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]",None
...,...,...,...,...
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]",None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]",None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_80f9b3d2-47b8-4152-9e45-a26dcf173335,Injection sous-cutanée à faire une fois par jo...,pendant 30 jours,"[4, 7]",None


In [97]:
test = "jusqu'au 6/6/2022"
df_blabla = Date().transform_data(test)
df_blabla

,data,day,month,year,day_of_week,day_from_today
0,"[jusqu, 6, 6, 2022]",6,6,2022,99,99


In [99]:
Duration().implicit_duration_1(test, [6,2], df=True)

,CareDuration,CareDurationType,CareEnd
0,4,Days,6 juin


In [128]:
test = "jusqu'au 10 juin"

In [136]:
Duration().get_duration(test, [6,99], df=True)

,CareDuration,CareDurationType,CareEnd
0,NaN,NaN,10 juin


In [162]:
df

,translation,Duration,Date,test,test_2
fichier,,,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[4, 19]",None,None
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]",None,None
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]",None,None
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None,None,None
...,...,...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]",None,None
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]",None,None
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]",None,None


In [193]:
df_temp = df.copy()

In [194]:
df_temp = Duration().apply_to_df(df_temp)

In [200]:
df_temp = df_temp.dropna()

In [201]:
df_temp

,translation,Duration,Date,temp
fichier,,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]","{'CareDuration': ['1'], 'CareDurationType': ['..."
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]","{'CareDuration': ['6'], 'CareDurationType': ['..."
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]","{'CareDuration': [3], 'CareDurationType': ['Da..."
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]","{'CareDuration': ['15'], 'CareDurationType': [..."
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]","{'CareDuration': [nan], 'CareDurationType': [n..."
...,...,...,...,...
2d035c4b-cdfa-4982-87dc-916fe07a0824_558bf9c3-a9e5-408e-bc55-da9844e1b61a,Ordonnance de renouvellement pour monsieur Did...,pour 3 mois,"[6, 29]","{'CareDuration': ['3'], 'CareDurationType': ['..."
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]","{'CareDuration': ['8'], 'CareDurationType': ['..."
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]","{'CareDuration': ['1'], 'CareDurationType': ['..."


In [206]:
for i in range(100):
    sentence = df_temp.iloc[i,0]
    if re.search("jusqu\'au\s", sentence) is not None:
        print(df_temp.iloc[i,1])
        print(df_temp.iloc[i,2])
        print(df_temp.iloc[i,3])

pour 6 jours
[4, 17]
{'CareDuration': ['6'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}
jusqu'au lundi 17 janvier 2022
[1, 14]
{'CareDuration': [3], 'CareDurationType': ['Days'], 'CareEnd': ['17 janvier']}
jusqu'au 8/12/2019
[99, 25]
{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['8 décembre']}
jusqu'au 28 novembre
[11, 23]
{'CareDuration': [5], 'CareDurationType': ['Days'], 'CareEnd': ['28 novembre']}
3 mois
[12, 11]
{'CareDuration': ['3'], 'CareDurationType': ['Months'], 'CareEnd': [nan]}
jusqu'au dimanche 7
[3, 4]
{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': [nan]}
jusqu'au 20 février
[2, 12]
{'CareDuration': [8], 'CareDurationType': ['Days'], 'CareEnd': ['20 février']}
jusqu'au 4 mars
[2, 21]
{'CareDuration': [11], 'CareDurationType': ['Days'], 'CareEnd': ['4 mars']}
jusqu'au 27 octobre
[9, 21]
{'CareDuration': [36], 'CareDurationType': ['Days'], 'CareEnd': ['27 octobre']}
jusqu'au 17 janvier
[1, 7]
{'CareDuration': [10], 'CareDurationType': ['

In [210]:
df_bb = Duration("../models/model_full/model-best").get_data()

In [211]:
df_bb

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[2, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"
